### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
32,42671083,15346,79085,3,0,0,1143,6,2025-06-06 19:33,2025-06-06 19:34,2,0,9102,6,1
33,42671099,144553,79085,51530,0,0,1144,0,2025-06-07 18:30,2025-06-07 18:30,1,1,8115,2,1
34,42675761,79085,142693,1002,0,74,1144,1,2025-06-08 21:59,2025-06-08 21:59,3,0,7304,4,1
35,42677948,116340,79085,51530,0,0,1144,2,2025-06-09 20:30,2025-06-09 20:30,3,2,3811,3,1
36,42674580,76562,79085,51531,0,0,1144,3,2025-06-10 09:30,2025-06-10 09:30,7,1,5988,2,1
37,42677914,116181,79085,3,0,0,1144,4,2025-06-11 18:25,2025-06-11 18:26,0,2,9120,5,1
38,42680920,58819,79085,51530,0,0,1144,5,2025-06-12 09:35,2025-06-12 09:35,1,0,7107,4,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,15346,22973,36096,1,18,4,1,0,532d2,11,9,8,42671083
1,79085,46985,33862,0,1,1,0,0,5-4-1a,7,9,8,42671083
0,144553,29351,38401,0,8,2,0,0,fff11,11,10,12,42671099
1,79085,40980,31930,0,3,2,0,0,5-3-2c,9,10,10,42671099
0,79085,20775,39897,0,15,1,0,0,4-2-4a,10,10,10,42675761
1,142693,49254,30132,0,2,0,0,0,T1PANTANAL,6,6,8,42675761
0,116340,29086,35412,2,8,0,0,0,NO TOCAR 1.0,10,11,9,42677948
1,79085,40876,34550,4,4,4,0,0,5-2-3f,10,10,10,42677948
0,76562,27705,33522,5,20,0,0,0,Hardangen.B2P,13,10,12,42674580
1,79085,42414,36597,0,4,5,4,0,3-5-2aSL,9,9,10,42674580


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1796

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,6276,0,0,0,0,0,0,0,12,1349
1,23775,0,0,0,0,0,0,0,11,1349
2,124139,0,0,0,0,0,0,0,10,1349
3,23755,0,0,0,0,0,0,0,9,1349
4,122169,0,0,0,0,0,0,0,8,1349
5,78671,0,0,0,0,0,0,0,7,1349
6,79085,0,0,0,0,0,0,0,6,1349
7,79866,0,0,0,0,0,0,0,5,1349
8,89569,0,0,0,0,0,0,0,4,1349
9,78686,0,0,0,0,0,0,0,3,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()